In [2]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [3]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

In [5]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [ ]:
bucket_name = 'wiki_ass3'
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if b.name != 'graphframes.sh':
        paths.append(full_path+b.name)
parquetFile = spark.read.parquet(*paths)

In [ ]:
doc_text_pairs = parquetFile.select("text", "id").rdd
doc_title_pairs = parquetFile.select("title", "id").rdd

In [14]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

# PLACE YOUR CODE HERE
def word_count(text):
  tokens = [token.group() for token in RE_WORD.finditer(text.lower()) if token not in all_stopwords]
  return len(tokens)

In [ ]:
word_counts = doc_text_pairs.map(lambda x: (x[1],word_count(x[0])))
DL = word_counts.collectAsMap()
doc_title = doc_title_pairs.map(lambda x: (x[1],x[0]))
DT = doc_title.collectAsMap()

In [ ]:
dl_file = 'dl.pkl'
with open(dl_file,'wb') as f:
    pickle.dump(DL,f)

bucket = client.bucket(bucket_name)
blob = bucket.blob(f"dl/{dl_file}")
blob.upload_from_filename(dl_file)

dt_file = 'dt.pkl'
with open(dt_file,'wb') as f:
    pickle.dump(DT,f)

bucket = client.bucket(bucket_name)
blob = bucket.blob(f"dt/{dt_file}")
blob.upload_from_filename(dt_file)